In [1]:
import pandas as pd
from datetime import datetime, timedelta
version_in = 'V0_1'
version_out = 'V0_3'
path_gen = "/home/pol/Escritorio/INVERBIS/2024/CancerColon/"
path_1 = path_gen + "Data/processed_data/CancerColon_"+str(version_in)+".csv"
colon = pd.read_csv(path_1, sep= ';')

In [2]:
print(colon.shape)
colon.columns

(13572, 34)


Index(['NASI seudonimizado', 'Subg. químico terapéutico ATC disp',
       'Principio activo ATC disp', 'Prezo PVP nomen disp', 'FECHA',
       'FECHA_FIN', 'Actividad', 'Hospital', 'Motivo saída', 'PAC',
       'Lugar asistencia', 'Motivo asistencia', 'Cód. CIAP 2', 'Desc. CIAP 2',
       'Orixe', 'Lista diagnósticos', 'Lista procedementos', 'Prioridade',
       'GNA', 'Cola', 'Tipo intervención', 'Orixe PM/Diagnóstico',
       'PM/Diagnóstico', 'Cód. cat. CIE', 'Categoría CIE', 'Subcategoría CIE',
       'CENTRO', 'SERVICIO', 'MEDICO', 'TIPO MUESTRA', 'MUESTRA',
       'DESCRIPCION', 'SNOMED', 'DATA EXTRACCIÓN'],
      dtype='object')

In [3]:
colon['Actividad'].unique()

array(['Ingreso Hospitalizacion', 'Cirugia', 'Inicio episodio',
       'Alta Hospitalizacion', 'Lista de Espera quirurgica - Inclusion',
       'Dispensación Farmacia Oncologica',
       'Asistencia a punto de Atención Continuada', 'Defunción',
       'Prueba laboratorio (APA)'], dtype=object)

In [ ]:
def filter_dataApa(trace, value):
    trace['FECHA'] = pd.to_datetime(trace['FECHA'], format = '%d-%m-%Y %H:%M:%S')
    # Fecha límite para el filtrado, también como datetime
    limit_date = pd.to_datetime(value, format= '%d-%m-%Y %H:%M:%S')

    limit_date_minus_10s = limit_date - pd.Timedelta(seconds=10)
    trace_filtered = trace[trace['FECHA'] >= limit_date_minus_10s] 
    print(value)
    print(trace_filtered['Actividad'].unique())
    print('\n')
    return trace_filtered


def crear_dead_bolean(trace):
    trace['Defuncion'] = True
    return trace


def modify_dead_date(trace):
    max_date = trace['FECHA'].max()
    new_date = max_date + timedelta(days=30)
    new_date = pd.to_datetime(new_date, format= '%d-%m-%Y %H:%M:%S')
    # Cambiar las fechas asociadas a 'Defunción'
    trace.loc[trace['Actividad'] == 'Defunción', ['FECHA', 'FECHA_FIN']] = new_date
    return trace

In [ ]:
#recorrer traza a traza
# y para cada traza: mirar la fecha de extracción de ese paciente
#                  -> filtrar filas de ese trace superiores a esa fecha (DATA_EXTRACT)


colon['FECHA'] = pd.to_datetime(colon['FECHA'], format= '%d-%m-%Y %H:%M:%S')
colon['FECHA_FIN'] = pd.to_datetime(colon['FECHA_FIN'], format= '%d-%m-%Y %H:%M:%S')
grouper = colon.groupby('NASI seudonimizado')

dates, traces = [], []
dates_dict = {}
for name, trace in grouper:
    trace['Defuncion'] = False
    if 'Prueba laboratorio (APA)' in trace['Actividad'].unique().tolist():
        print(name)
        df = trace.query(" Actividad == 'Prueba laboratorio (APA)'")
        value = df['FECHA'].values[0]
        dates.append(value)
        dates_dict[name] = value
        
        #!boleano defunciones
        if 'Defunción' in trace['Actividad'].unique().tolist():
            trace = crear_dead_bolean(trace)
            trace = modify_dead_date(trace)
            
        print('FILTRADO!!!\n')

        traces.append(filter_dataApa(trace, value))

colon_filtered = pd.concat(traces)

In [ ]:
list(dates_dict.keys())
name = '0067638e545bc471e1f89de1738c1d5c'

print(colon['NASI seudonimizado'].nunique())
print(colon_filtered['NASI seudonimizado'].nunique())
print(colon_filtered.columns)
colon_filtered.groupby('Actividad')['NASI seudonimizado'].nunique()

In [ ]:
id = '010b7970230a0be9031b74e8aff8df0f'
colon_prueba = colon[colon['NASI seudonimizado'] == id]
colon_prueba

In [ ]:
id = '010b7970230a0be9031b74e8aff8df0f'
colon_filtered_prueba = colon_filtered[colon_filtered['NASI seudonimizado'] == id]
colon_filtered_prueba

In [ ]:
colon_filtered['Defuncion'].value_counts()

In [ ]:
colon_filtered['FECHA'].max()

In [ ]:
# Eliminar '\n' y '\r' en valores y cabeceras
colon_filtered = colon_filtered.replace({r'\r': '', r'\n': ''}, regex=True)

In [ ]:
colon_filtered.to_csv(path_gen+"Data/processed_data/CancerColon_basico_filtered"+str(version_out)+".csv", date_format="%d-%m-%Y %H:%M:%S", index=False,  encoding="UTF-8", sep = ';')